In [1]:
import numpy as np

In [17]:
def fit(x_train,y_train):
    result={}
    class_values=set(y_train)
    for current_value in class_values:
        result[current_value]={}
        result["total_data"]=len(y_train)
        class_current_row=(y_train==current_value)
        x_train_current=x_train[class_current_row]
        y_train_current=y_train[class_current_row]
        result[current_value]["total_count"]=len(y_train_current)
        num_features=x_train.shape[1]
        
        for j in range(1,num_features+1):
            result[current_value][j]={}
            all_possible_values=x_train[:,j-1]
            
            for value in all_possible_values:
                result[current_value][j][value]=(x_train_current[:,j-1]==value).sum()
    
    return result

In [47]:
def probablity(dict,x,current_class):
    output=np.log(dict[current_class]["total_count"])- np.log(dict["total_data"])
    num_features=(len(dict[current_class].keys()))-1
    for j in range(1,num_features+1):
        xj=x[j-1]
        count_current_class_with_xj=dict[current_class][j][xj]+1
        count_current_class=dict[current_class]["total_count"]+len(dict[current_class][j].keys())
        current_xj_probablity=np.log(count_current_class_with_xj)- np.log(count_current_class)
        output=output + current_xj_probablity
        
    return output

In [30]:
def predictsiglepoint(dict,x):
    classes =dict.keys()
    bestp=-1000
    best_class=-1
    first_run=True
    for current_class in classes:
        if current_class=="total_data":
            continue
        p_current_class=probablity(dict,x,current_class)
        if first_run or p_current_class>bestp:
            bestp=p_current_class
            best_class=current_class
        first_run=False
    return best_class

In [31]:
def predict(dict,x_test):
    y_pred=[]
    for x in x_test:
        x_class =predictsiglepoint(dict,x)
        y_pred.append(x_class)
    return y_pred

In [21]:
def makelabel(column):
    second_limit=column.mean()
    first_limit=0.5*second_limit
    third_limit=1.5*second_limit
    for i in range(0,len(column)):
        if column[i]<first_limit:
            column[i]=0
        elif column[i]<second_limit:
            column[i]=1        
        elif column[i]<third_limit:
            column[i]=2
        else:
            column[i]=3
            
            
    return column
            

In [22]:
from sklearn import datasets
iris=datasets.load_iris()
x=iris.data
y=iris.target

In [50]:
for i in range(0,x.shape[-1]):
    x[:,i]=makelabel(x[:,i])

In [25]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(x,y,random_state=1)

In [26]:
dictionary=fit(x_train,y_train)

In [48]:
y_pred=predict(dictionary,x_test)

In [49]:
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test,y_pred)

array([[13,  0,  0],
       [ 0, 16,  0],
       [ 0,  0,  9]], dtype=int64)